
### Example of Agrihealth use: 

**Using TiffHandler and ShapefileHandler**

Use TiffHandler to load and process your .tif raster image. From satellite or drone imagery with multiple spectral bands (e.g., red, NIR) needed for vegetation analysis.

**Use ShapefileHandler** to load your shapefile, which contains vector data such as field boundaries or management zones. This allows you to link raster-based vegetation indices to real-world geographic features.

**Calculating Vegetation Indices**

Upload your own .tif image and choose the vegetation index you want to calculate. The TiffHandler will automatically extract the required spectral bands and compute the selected index using built-in formulas.

The results will be returned as a DataFrame, which you can merge with shapefile data to assign vegetation index values to specific geographic areas.

**Plotting the Results**

Use matplotlib to visualize the shapefile polygons. Each polygon will be color-coded based on its vegetation index value, helping you quickly identify differences in vegetation health across your fields or zones.

**Interactive Map Visualization**

To explore your data interactively, use one of the following map classes:

*LeafletMap* – a lightweight option for basic interactive mapping.

*FoliumMap* – includes more advanced features like basemap toggling and layer controls.

Both map types support coloring your shapefile polygons by vegetation index values, giving you a clear, visual way to assess vegetation conditions across your area of interest.


### 1. Import necessary libraries from agrihealth

In [ ]:
from agrihealth import TiffHandler, LeafletMap, FoliumMap, ShapefileHandler


### 2. Load a TIF Image and Extract Bands

In [ ]:
# Provide the path to your .tif image
tif_path = "./data/fields.tif"

# Initialize TiffHandler
tiff = TiffHandler(tif_path)

# Load and extract bands
df = tiff.extract_bands()

# Show the first few rows
df.head()


### 3. Initialize the TiffHandler again for the specific .tif file and ShapefileHandler for the shapefile

In [ ]:
 # Initialize the TiffHandler and ShapefileHandler
tiff_handler = TiffHandler("./data/fields.tif")
shapefile_handler = ShapefileHandler('./data/fields/All_Fields_Export.shp')

In [ ]:
# Print the first few rows of the DataFrame (NDVI values in the DataFrame)
print(df.head())
print(df['NDVI'].describe())

In [ ]:
# Extract NDVI from the raster
df = tiff_handler.extract_bands()

# Get the shapefile data
gdf = shapefile_handler.get_shapefile_data()

# Print the GeoDataFrame (shapefile data)
print(gdf)


### 4. Visualization 

In [ ]:
# Import the necessary library for plotting
import matplotlib.pyplot as plt

# We merge the vegetation index (ndvi, evi, gndvi or savi) data into the shapefile data (GeoDataFrame)
gdf = shapefile_handler.get_shapefile_data()

# Merge NDVI values into the GeoDataFrame
gdf['NDVI'] = df['NDVI']  # Assuming df has NDVI values in the same order

# Plot the shapefile with NDVI values
fig, ax = plt.subplots(figsize=(10, 10))
gdf.plot(column='NDVI', ax=ax, legend=True,
         legend_kwds={'label': "NDVI by Polygon",
                      'orientation': "horizontal"})
ax.set_title("Shapefile with Vegetation Index (NDVI)")
plt.show()


### 5. Create Maps 

In [ ]:
# Compute the map center using geometries
centroid = gdf.geometry.centroid
map_center = [centroid.y.mean(), centroid.x.mean()]

# Initialize the map
fmap = LeafletMap(center=map_center, zoom=13)

# Add desired basemaps (each only once)
fmap.add_basemaps()

# Add vector layer colored by 'NDVI'
fmap.add_vector(gdf, color_column="NDVI")

# Display the map
fmap.show()

# Create map with Folium

In [ ]:
# Compute map center
centroid = gdf.geometry.centroid
map_center = [centroid.y.mean(), centroid.x.mean()]

# Initialize the map
fmap = FoliumMap(location=map_center, zoom_start=13)

# Add all basemaps (this will automatically avoid duplicates)
fmap.add_all_basemaps()  # This adds all basemaps defined in self.basemaps

# Add vector layer to the map (color based on 'NDVI' column)
fmap.add_vector(gdf, color_column='NDVI')

# Add layer control (to toggle between layers)
fmap.add_layer_control()

# Display the map
fmap.show()